In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, time, warnings, gzip, gc, random, math, shap, pickle, optuna, csv, sys
from IPython.display import display
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 400)

warnings.filterwarnings("ignore")

In [47]:
time0 = time.time()

colstouse = ['id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',    
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq']

df = pd.read_csv('../input/lending-club-20072020q1/Loan_status_2007-2020Q3.gzip', 
                 usecols=colstouse, skiprows=lambda i: i>0 and random.random() > 0.3, index_col = [0])
display(df.shape, df.head(), time.time()-time0)
gc.collect()

(877661, 57)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1076863,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,690,694,1.0,35.0,NaN,10.0,0.0,5598,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.0000,Jan-2015,357.48,NaN,Apr-2016,604,600,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1075269,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,Dec-2011,Fully Paid,n,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,Nov-2004,730,734,3.0,NaN,NaN,9.0,0.0,7963,28.3%,12.0,f,0.0,0.0,5632.210000,5632.21,5000.00,632.21,0.00,0.00,0.0000,Jan-2015,161.03,NaN,Feb-2017,564,560,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1069639,7000,7000,7000.0,60 months,15.96%,170.08,C,C5,Southern Star Photography,8 years,RENT,47004.0,Not Verified,Dec-2011,Fully Paid,n,debt_consolidation,Loan,280xx,NC,23.51,0.0,Jul-2005,690,694,1.0,NaN,NaN,7.0,0.0,17726,85.6%,11.0,f,0.0,0.0,10137.840008,10137.84,7000.00,3137.84,0.00,0.00,0.0000,May-2016,1313.76,NaN,Nov-2019,654,650,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1064687,9000,9000,9000.0,36 months,13.49%,305.38,C,C1,Va. Dept of Conservation/Recreation,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,debt_consolidation,freedom,245xx,VA,10.08,0.0,Apr-2004,710,714,1.0,NaN,NaN,4.0,0.0,10452,91.7%,9.0,f,0.0,0.0,2277.320000,2277.32,1256.14,570.26,0.00,450.92,4.1600,Jul-2012,305.38,NaN,Oct-2016,619,615,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1069057,10000,10000,10000.0,36 months,10.65%,325.74,B,B2,SFMTA,3 years,RENT,100000.0,Source Verified,Dec-2011,Charged Off,n,other,Other Loan,951xx,CA,7.06,0.0,May-1991,720,724,2.0,NaN,NaN,14.0,0.0,11997,55.5%,29.0,f,0.0,0.0,7471.990000,7471.99,5433.47,1393.42,0.00,645.10,6.3145,Oct-2013,325.74,NaN,Oct-2016,654,650,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0


17.85705590248108

27

In [48]:
display(sys.getsizeof(df)/1048576)
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
display(sys.getsizeof(df)/1048576)

df.term = pd.to_numeric(df.term.str.rstrip(' months'))
df.int_rate = pd.to_numeric(df.int_rate.str.rstrip('%'))
df.revol_util = pd.to_numeric(df.revol_util.str.rstrip('%'))
display(sys.getsizeof(df)/1048576)

dftypes = df.dtypes
o_features = list((dftypes[dftypes=='O']).index)
for col in o_features:
    df[col] = df[col].astype('category')
display(sys.getsizeof(df)/1048576, df.dtypes, df.head())

1429.5330896377563

1342.484803199768

1201.3264045715332

219.61736965179443

loan_amnt                         int64
funded_amnt                       int64
funded_amnt_inv                 float32
term                              int64
int_rate                        float64
installment                     float32
grade                          category
sub_grade                      category
emp_title                      category
emp_length                     category
home_ownership                 category
annual_inc                      float32
verification_status            category
issue_d                        category
loan_status                    category
pymnt_plan                     category
purpose                        category
title                          category
zip_code                       category
addr_state                     category
dti                             float32
delinq_2yrs                     float32
earliest_cr_line               category
fico_range_low                    int64
fico_range_high                   int64


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1076863,10000,10000,10000.0,36,13.49,339.309998,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,690,694,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12231.889648,12231.889648,10000.000000,2214.919922,16.969999,0.000000,0.0000,Jan-2015,357.480011,NaN,Apr-2016,604,600,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1075269,5000,5000,5000.0,36,7.90,156.460007,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,Dec-2011,Fully Paid,n,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,Nov-2004,730,734,3.0,NaN,NaN,9.0,0.0,7963,28.3,12.0,f,0.0,0.0,5632.209961,5632.209961,5000.000000,632.210022,0.000000,0.000000,0.0000,Jan-2015,161.029999,NaN,Feb-2017,564,560,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1069639,7000,7000,7000.0,60,15.96,170.080002,C,C5,Southern Star Photography,8 years,RENT,47004.0,Not Verified,Dec-2011,Fully Paid,n,debt_consolidation,Loan,280xx,NC,23.51,0.0,Jul-2005,690,694,1.0,NaN,NaN,7.0,0.0,17726,85.6,11.0,f,0.0,0.0,10137.839844,10137.839844,7000.000000,3137.840088,0.000000,0.000000,0.0000,May-2016,1313.760010,NaN,Nov-2019,654,650,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1064687,9000,9000,9000.0,36,13.49,305.380005,C,C1,Va. Dept of Conservation/Recreation,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,debt_consolidation,freedom,245xx,VA,10.08,0.0,Apr-2004,710,714,1.0,NaN,NaN,4.0,0.0,10452,91.7,9.0,f,0.0,0.0,2277.320068,2277.320068,1256.140015,570.260010,0.000000,450.920013,4.1600,Jul-2012,305.380005,NaN,Oct-2016,619,615,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0
1069057,10000,10000,10000.0,36,10.65,325.739990,B,B2,SFMTA,3 years,RENT,100000.0,Source Verified,Dec-2011,Charged Off,n,other,Other Loan,951xx,CA,7.06,0.0,May-1991,720,724,2.0,NaN,NaN,14.0,0.0,11997,55.5,29.0,f,0.0,0.0,7471.990234,7471.990234,5433.470215,1393.420044,0.000000,645.099976,6.3145,Oct-2013,325.739990,NaN,Oct-2016,654,650,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0


In [49]:
df.shape

(877661, 57)

In [50]:
df.to_pickle('LCLoans_57_800k.pkl')

In [52]:
os.listdir()

['__notebook_source__.ipynb', 'LCLoans_57_800k.pkl', '.virtual_documents']